In [1]:
import http.server
import socketserver
import base64
from urllib.parse import urlparse, parse_qs
from datetime import datetime
import threading

PORT = 8080
LOG_FILE = "captured_data.txt"

class RequestHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        # Parse the URL and parameters
        parsed = urlparse(self.path)
        params = parse_qs(parsed.query)
        
        # Get the base64 data
        b64_data = params.get('b', [''])[0]
        
        if b64_data:
            try:
                # Decode the base64 data
                decoded = base64.b64decode(b64_data).decode('utf-8')
                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                
                # Log to file and console
                log_entry = f"[{timestamp}] {decoded}\n"
                with open(LOG_FILE, 'a', encoding='utf-8') as f:
                    f.write(log_entry)
                print(log_entry.strip())
                
            except Exception as e:
                print(f"Error decoding: {e}")
        
        # Send response
        self.send_response(200)
        self.send_header('Content-type', 'text/plain')
        self.end_headers()
        self.wfile.write(b"OK")
    
    def log_message(self, format, *args):
        # Suppress default logging
        return

def start_server():
    print(f"Starting HTTP listener on port {PORT}")
    print(f"Logging to: {LOG_FILE}")
    print("Server is running...")
    
    with socketserver.TCPServer(("0.0.0.0", PORT), RequestHandler) as httpd:
        try:
            httpd.serve_forever()
        except KeyboardInterrupt:
            print("\nStopping server...")

# Clear previous log file
open(LOG_FILE, 'w').close()

# Start server in a thread so Jupyter doesn't block
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

print("Server started in background. Check the terminal/console for incoming data.")
print(f"Your data will be saved to: {LOG_FILE}")

Starting HTTP listener on port 8080Server started in background. Check the terminal/console for incoming data.
Your data will be saved to: captured_data.txt

Logging to: captured_data.txt
Server is running...


[2025-10-24 21:16:44] FILE_START: product.cfg (path: /pref/product.cfg)
[2025-10-24 21:16:45] [product.cfg] LINE:1: ### read only ###
[2025-10-24 21:16:46] [product.cfg] LINE:2: BRAND=Drift
[2025-10-24 21:16:46] [product.cfg] LINE:3: MODEL=GHOST X
[2025-10-24 21:16:47] [product.cfg] LINE:4:
[2025-10-24 21:16:47] [product.cfg] LINE:5: ################################################
[2025-10-24 21:16:47] [product.cfg] LINE:6:
[2025-10-24 21:16:48] [product.cfg] LINE:7: wifi_on=1
[2025-10-24 21:16:48] [product.cfg] LINE:8: Fn=setting
[2025-10-24 21:16:49] [product.cfg] LINE:9: auto_off=0
[2025-10-24 21:16:49] [product.cfg] LINE:10: lang=en
[2025-10-24 21:16:50] [product.cfg] LINE:11: photo_tl=0
[2025-10-24 21:16:50] [product.cfg] LINE:12: caf=1
[2025-10-24 21:16:51] [product.cfg] LINE:13: hdmi_osd=1
[2025-10-24 21:16:51] [product.cfg] LINE:14: F2=onoff_lcd
[2025-10-24 21:16:52] [product.cfg] LINE:15: photo_tl_interval=5
[2025-10-24 21:16:52] [product.cfg] LINE:16: photo_tl_num=100
[2025-

Client side, run this:

cat > /tmp/send_to_jupyter.sh << 'EOF'
#!/bin/sh

# Your Windows desktop IP
SERVER="192.168.1.166"
PORT=8080
TEST_LIMIT=-1

echo "=== SENDING FILES TO JUPYTER NOTEBOOK ==="

# Find all human-readable files
for f in $(find /pref /etc /tmp /usr/local /tmp/SD0 /tmp/FL0 -type f \( -name "*.conf" -o -name "*.cfg" -o -name "*.sh" -o -name "*.txt" -o -name "*.log" -o -name "*.json" -o -name "*.ini" \) 2>/dev/null | head -100); do
    [ -f "$f" ] || continue
    bn=$(basename "$f")
    echo "=== Processing file: $bn ==="

    # Send file header with full path
    HEADER="FILE_START: $bn (path: $f)"
    B64=$(printf '%s' "$HEADER" | base64 | tr -d '\n')
    B64SAFE=$(printf '%s' "$B64" | sed -e 's/+/%%2B/g' -e 's/\//%%2F/g' -e 's/=/%%3D/g')
    printf 'GET /?b=%s HTTP/1.0\r\nHost: %s\r\n\r\n' "$B64SAFE" "$SERVER" | nc -w 5 "$SERVER" "$PORT"
    sleep 0.5

    # Count total lines
    total_lines=$(wc -l < "$f" 2>/dev/null || echo 0)
    echo "File has $total_lines lines"

    n=0
    while IFS= read -r line || [ -n "$line" ]; do
        n=$((n + 1))
        if [ "$TEST_LIMIT" -ge 0 ] && [ "$n" -gt "$TEST_LIMIT" ]; then
            echo "(test limit reached for $bn)"
            break
        fi

        # Send line with number and file identification
        line_with_number="[$bn] LINE:$n: $line"
        printf '[%d/%d] %s\n' "$n" "$total_lines" "$line"
        B64=$(printf '%s' "$line_with_number" | base64 | tr -d '\n')
        B64SAFE=$(printf '%s' "$B64" | sed -e 's/+/%%2B/g' -e 's/\//%%2F/g' -e 's/=/%%3D/g')
        printf 'GET /?b=%s HTTP/1.0\r\nHost: %s\r\n\r\n' "$B64SAFE" "$SERVER" | nc -w 5 "$SERVER" "$PORT"

        # Progress indicator and delay to ensure all lines are received
        if [ $((n % 5)) -eq 0 ]; then
            echo "  Progress: $n/$total_lines"
            sleep 0.05
        else
            sleep 0.05
        fi
    done < "$f"

    # Send file footer
    FOOTER="FILE_END: $bn (sent $n/$total_lines lines)"
    B64=$(printf '%s' "$FOOTER" | base64 | tr -d '\n')
    B64SAFE=$(printf '%s' "$B64" | sed -e 's/+/%%2B/g' -e 's/\//%%2F/g' -e 's/=/%%3D/g')
    printf 'GET /?b=%s HTTP/1.0\r\nHost: %s\r\n\r\n' "$B64SAFE" "$SERVER" | nc -w 5 "$SERVER" "$PORT"

    echo "=== Finished $bn (sent $n lines) ==="
    sleep 1
done

# Send completion marker
FINAL="=== ALL FILES COMPLETED ==="
B64=$(printf '%s' "$FINAL" | base64 | tr -d '\n')
B64SAFE=$(printf '%s' "$B64" | sed -e 's/+/%%2B/g' -e 's/\//%%2F/g' -e 's/=/%%3D/g')
printf 'GET /?b=%s HTTP/1.0\r\nHost: %s\r\n\r\n' "$B64SAFE" "$SERVER" | nc -w 5 "$SERVER" "$PORT"

echo "=== ALL FILES SENT TO JUPYTER ==="
EOF

chmod +x /tmp/send_to_jupyter.sh
sh /tmp/send_to_jupyter.sh